In [26]:
import duckdb
import pandas as pd

In [6]:

sql_query_import_1 = '''
CREATE OR REPLACE TABLE parking_violation_codes AS
SELECT * 
FROM read_csv_auto(
    'data/dof_parking_violation_codes.csv',
    normalize_names=True
    )

'''

sql_query_import_2 = '''
CREATE OR REPLACE TABLE parking_violations_2023 AS
SELECT * 
FROM read_csv_auto(
    'data/parking_violations_issued_fiscal_year_2023_sample.csv',
    normalize_names=True
    )

'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    con.sql(sql_query_import_1)
    con.sql(sql_query_import_2)



,name
0,parking_violation_codes
1,parking_violations_2023


In [8]:
sql_query = '''
SELECT * from parking_violation_codes LIMIT 5
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,code,definition,manhattan_96th_st_below,all_other_areas
0,1,FAILURE TO DISPLAY BUS PERMIT,515,515
1,2,NO OPERATOR NAM/ADD/PH DISPLAY,515,515
2,3,UNAUTHORIZED PASSENGER PICK-UP,515,515
3,4,BUS PARKING IN LOWER MANHATTAN,115,115
4,5,BUS LANE VIOLATION,250,250
5,6,OVERNIGHT TRACTOR TRAILER PKG,500,500
6,7,FAILURE TO STOP AT RED LIGHT,50,50
7,8,IDLING,115,115
8,9,OBSTRUCTING TRAFFIC/INTERSECT,115,115
9,10,NO STOPPING-DAY/TIME LIMITS,115,115


In [35]:
sql_query = '''
show tables
''' 

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,name
0,bronze_parking_violation_codes
1,bronze_parking_violations
2,first_model
3,parking_violation_codes
4,parking_violations_2023
5,ref_model
6,silver_parking_violation_codes
7,silver_parking_violations
8,silver_violation_tickets
9,silver_violation_vehicles


In [45]:
sql_query = '''
SELECT * FROM silver_parking_violation_codes
''' 

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,violation_code,definition,is_manhattan_96th_st_below,fee_usd
0,1,FAILURE TO DISPLAY BUS PERMIT,True,515
1,1,FAILURE TO DISPLAY BUS PERMIT,False,515
2,2,NO OPERATOR NAM/ADD/PH DISPLAY,True,515
3,2,NO OPERATOR NAM/ADD/PH DISPLAY,False,515
4,3,UNAUTHORIZED PASSENGER PICK-UP,True,515
...,...,...,...,...
189,97,VACANT LOT,False,45
190,98,OBSTRUCTING DRIVEWAY,True,95
191,98,OBSTRUCTING DRIVEWAY,False,95
192,99,OTHER,True,105


In [55]:
sql_query = '''
SELECT * FROM silver_violation_tickets limit 3
''' 

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,summons_number,issue_date,violation_code,is_manhattan_96th_st_below,issuing_agency,violation_location,violation_precinct,issuer_precinct,issuer_code,issuer_command,issuer_squad,violation_time,violation_county,violation_legal_code,fee_usd
0,9010912681,2022-10-11,17,False,T,50.0,50,50,365910,T201,D,0825A,BX,NaN,95
1,4858762841,2023-08-21,36,False,V,NaN,0,0,0,None,None,1036A,BK,True,50
2,4854645684,2023-07-26,36,False,V,NaN,0,0,0,None,None,0126P,QN,True,50


In [54]:
sql_query1 = '''
SELECT SUM(total_revenue_usd) FROM gold_ticket_metrics
''' 

sql_query2 = '''
SELECT * FROM gold_vehicles_metrics limit 50
''' 

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query1).df())


with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query2).df())

,sum(total_revenue_usd)
0,7273820.0


,registration_state,ticket_count
0,NJ,9258
1,PA,3514
2,FL,2414
3,CT,1787
4,GA,840
5,VA,797
6,MA,788
7,IN,765
8,NC,614
9,TX,594


In [49]:
sql_query = '''
WITH manhattan_violation_codes AS (
    SELECT
        violation_code,
        definition,
        TRUE AS is_manhattan_96th_st_below,
        manhattan_96th_st_below AS fee_usd,
    FROM
        bronze_parking_violation_codes
),

all_other_violation_codes AS (
    SELECT
        violation_code,
        definition,
        FALSE AS is_manhattan_96th_st_below,
        all_other_areas AS fee_usd,
    FROM
        bronze_parking_violation_codes
)

SELECT * FROM manhattan_violation_codes where violation_code=97
UNION ALL
SELECT * FROM all_other_violation_codes  where violation_code=97
ORDER BY violation_code ASC
''' 

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,violation_code,definition,is_manhattan_96th_st_below,fee_usd
0,97,VACANT LOT,True,65
1,97,VACANT LOT,False,45


In [ ]:
sql_query = '''
  select * from "nyc_parking_violations"."main_dbt_test__audit"."violation_codes_revenue"
''' 

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())